In [1]:
import sys
import os
from collections import Counter 

sys.path.append("/home/vib9/src/voxelmorph/pytorch/")
import datagenerators

sys.path.append("/home/vib9/src/SL-Net/superlayer/voxel")

import numpy as np

import torch
import torch.nn as nn
from torch import optim
from torch.utils.data import DataLoader, random_split

sys.path.append("../../")
sys.path.append("../../scripts/")
import superlayer.utils

dir_img = '/home/gid-dalcaav/projects/neuron/data/t1_mix/proc/resize256-crop_x32-slice100/train/vols/'
dir_mask = '/home/gid-dalcaav/projects/neuron/data/t1_mix/proc/resize256-crop_x32-slice100/train/asegs/'

dir_train = '/home/vib9/src/SL-Net/jupyter/partitions/train.txt'
dir_val = '/home/vib9/src/SL-Net/jupyter/partitions/val.txt'
dir_scarce = '/home/vib9/src/SL-Net/jupyter/partitions/scarce_train.txt'

In [2]:
dir_i = '/home/gid-dalcaav/projects/neuron/data/t1_mix/proc/resize256-crop_x32-slice100/train/vols/'
atlas_file = '/home/vib9/src/voxelmorph/data/atlas_norm.npz'
train_file = '/home/vib9/src/SL-Net/jupyter/partitions/train.txt'
train_file = open(train_file)
train_strings = train_file.readlines()
#train_vol_names = [(dir_i + x.strip() + ".npz") for x in train_strings]
train_vol_names = [(dir_i + x.strip() + ".npz").replace("vols", "asegs").replace("norm","aseg") for x in train_strings]

def get_dist(atlas_file, data_dir, train_file, train_vol_names, num_samples):
    
    atlas = np.load(atlas_file)
    atlas_seg = np.concatenate(atlas['seg'][:,:,100]).tolist()
    atlas_set = set(atlas_seg)
    atlas_dict = {i:atlas_seg.count(i) for i in atlas_set}
    agr_dict = Counter({})
    
    for i in range(num_samples):
        vol = np.concatenate(np.load(train_vol_names[i])['vol_data']).tolist()
        
        num_set = set(vol)
        temp_dict = Counter({i:vol.count(i) for i in num_set})
        agr_dict = agr_dict + temp_dict
            
    atlas_dict = sorted(atlas_dict.items(), key=lambda x: x[1], reverse=True)
    agr_dict = sorted(dict(agr_dict).items(), key=lambda x: x[1], reverse=True)
    
    return atlas_dict, agr_dict

In [3]:
atlas_dict, agr_dict = get_dist(atlas_file=atlas_file, 
                                 data_dir=dir_i, 
                                 train_file=train_file, 
                                 train_vol_names=train_vol_names, 
                                 num_samples=len(train_vol_names))

In [4]:
print("target num: " + str(160 * 192))

target num: 30720


In [5]:
def get_num(dic):
    s = 0
    for key in dic:
        s += key[1]
    return s

In [6]:
atlas_dict

[(0.0, 17663),
 (2.0, 2987),
 (41.0, 2696),
 (42.0, 2456),
 (3.0, 2205),
 (16.0, 1344),
 (10.0, 283),
 (49.0, 245),
 (4.0, 142),
 (43.0, 132),
 (17.0, 107),
 (53.0, 106),
 (60.0, 102),
 (28.0, 101),
 (24.0, 80),
 (14.0, 18),
 (31.0, 15),
 (63.0, 15),
 (5.0, 7),
 (46.0, 7),
 (7.0, 4),
 (44.0, 3),
 (15.0, 2)]

In [7]:
get_num(atlas_dict)

30720

In [8]:
agr_dict

[(0.0, 110240067),
 (41.0, 14325880),
 (2.0, 14272365),
 (42.0, 12036083),
 (3.0, 11670197),
 (16.0, 6340677),
 (10.0, 1920827),
 (49.0, 1779881),
 (4.0, 1112405),
 (43.0, 1086074),
 (28.0, 920685),
 (60.0, 869434),
 (17.0, 696217),
 (53.0, 683989),
 (24.0, 679003),
 (14.0, 234998),
 (63.0, 209125),
 (31.0, 188392),
 (77.0, 163255),
 (252.0, 137467),
 (11.0, 124622),
 (5.0, 119548),
 (50.0, 103251),
 (44.0, 86096),
 (251.0, 66952),
 (46.0, 33362),
 (12.0, 31474),
 (51.0, 18719),
 (7.0, 10767),
 (15.0, 4234),
 (8.0, 3601),
 (47.0, 2276),
 (80.0, 292),
 (52.0, 213),
 (13.0, 144),
 (253.0, 108),
 (254.0, 87),
 (30.0, 14),
 (54.0, 12),
 (62.0, 6),
 (255.0, 1)]

In [9]:
get_num(agr_dict)

180172800